In [ ]:
from neo4j.util import watch
import logging
from sys import stdout

watch("neo4j.bolt", logging.DEBUG, stdout)


In [1]:
# http://pro.arcgis.com/en/pro-app/tool-reference/analysis/generate-near-table.htm
# http://pro.arcgis.com/en/pro-app/arcpy/get-started/data-access-using-cursors.htm

import arcpy
from neo4j.v1 import GraphDatabase


arcpy.env.workspace = "./sandbox_data/Canada"

in_features = "earthquakes_location_2010_20.shp"
near_features = "Pipelines_Segments_Permited.shp"
out_table = "earthquakes_THREATS_pipelines.dbf"

search_radius = '1500 Meters'
location = 'NO_LOCATION'
angle = 'NO_ANGLE'
closest = 'ALL'
closest_count = 3

print("Start processing ...")

arcpy.GenerateNearTable_analysis(in_features, [near_features], out_table, search_radius, 
                                 location, angle, closest, closest_count)



driver = GraphDatabase.driver("bolt://54.201.105.15:7687", auth=("geo2graph", "geo2graph"))


cursor = arcpy.SearchCursor(in_features,
                            where_clause="magnitude  >= 3",
                            fields="FID; depth; magnitude")
for row in cursor:
    create_earthquake_cypher = "CREATE (e:Threat_Mechanism " \
                               "{fid:$fid, depth:$depth, magnitude:$magnitude} )"
        
    with driver.session() as session:
        fid = row.getValue("FID")
        depth = row.getValue("depth")
        magnitude = row.getValue("magnitude")
        
        session.run(create_earthquake_cypher, fid=fid, depth=depth, magnitude=magnitude)
        

        
cursor = arcpy.SearchCursor(near_features,
                            fields="FID; PROJECT_NU; SEGMENT_NU; LINE_TYPE_; STATUS; PROPONENT")
for row in cursor:
    create_pipeline_cypher = "CREATE (p:Pipeline " \
                             "{fid:$fid, project_id:$project_id, segment_id:$segment_id, " \
                             " line_type:$line_type, status:$status, operator:$operator} )"
        
    with driver.session() as session:
        fid = row.getValue("FID")
        project_id = row.getValue("PROJECT_NU")
        segment_id = row.getValue("SEGMENT_NU")
        line_type = row.getValue("LINE_TYPE_")
        status = row.getValue("STATUS")
        operator = row.getValue("PROPONENT")
        
        session.run(create_pipeline_cypher, fid=fid, project_id=project_id, 
                    segment_id=segment_id, line_type=line_type, status=status, 
                    operator=operator)


        
cursor = arcpy.SearchCursor(out_table)
for row in cursor:
    create_threat_cypher = "MATCH (e:Threat_Mechanism {fid:$e_fid}) " \
                           "MATCH (p:Pipeline {fid:$p_fid}) " \
                           "CREATE (e)-[:Threats {distance:$distance}]->(p)"
        
    with driver.session() as session:
        e_fid = row.getValue("IN_FID")
        p_fid = row.getValue("NEAR_FID")
        distance = row.getValue("NEAR_DIST")
        
        session.run(create_threat_cypher, e_fid=e_fid, p_fid=p_fid, distance=distance)


driver.session().run("MATCH (n:Threat_Mechanism) WHERE size((n)--())=0 DELETE (n)")

driver.close()

print("Done")

Start processing ...
Done
